In [ ]:
import os
import sys

**Specify the locations of the cloned [single-cell-curation repo](https://github.com/chanzuckerberg/single-cell-curation) & your API key file**

In [ ]:
scc_repo_loc = os.path.expanduser('~/GitClones/CZI/')
api_key_file_path = os.path.expanduser('~/Documents/keys/cxg-api-key.txt')

In [ ]:
sys.path.append(os.path.abspath(scc_repo_loc + 'single-cell-curation/notebooks/curation_api/python/'))


from src.utils.config import set_api_access_config
from src.dataset import create_dataset,upload_local_datafile
from src.collection import get_collection,create_revision

In [ ]:
set_api_access_config(api_key_file_path) #use env='dev' to work on the dev site

**Specify the Collection to upload to**<br>
If a Revision, use the Revision ID, not the Published ID

In [ ]:
collection_id = ''

**List the existing Datasets in the Collection**

In [ ]:
coll_datasets = {d['dataset_id']:d['title'] for d in get_collection(collection_id)['datasets']}
coll_datasets

**Set the directory that the files to upload are in**

In [ ]:
directory = os.path.expanduser('~/Downloads/')

**List certain files in that directory**<br>
The curation_qa notebook saves files with a `_revised.h5ad` suffix

In [ ]:
for f in os.listdir(directory):
    if f.endswith('_revised.h5ad'):
        print(f)

**Fill in file name:Dataset ID pairs**<br>
If adding a dataset, rather than replacing an existing dataset, use `new` for the Dataset ID

In [ ]:
datasets = {
    directory + 'my_matrix_revised.h5ad': 'new'
}

**Ensure that each file exists and each Dataset exists or is `new`**

In [ ]:
for f,d in datasets.items():
    if not os.path.exists(f):
        print('Invalid file:' + f)
    if d != 'new':
        if d not in coll_datasets:
            print('Invalid dataset:' + d)
        if list(datasets.values()).count(d) > 1:
            print('Repeated dataset:' + d)

**Upload each Dataset**

In [ ]:
for f,d in datasets.items():
    if d == 'new':
        d = create_dataset(collection_id)
    upload_local_datafile(f, collection_id, d)